In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import time

import seaborn as sns
import numpy as np

from torch.utils.data import  DataLoader
from torchvision import datasets, transforms

from dataset import ContrastiveDataset
from optimizer import LARS
from loss import NT_Xent
from model import ContrastiveLearningModel

sns.set_theme()

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
seed = 42 
np.random.seed(seed)
torch.manual_seed(seed)

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

gpu = torch.cuda.get_device_properties(0)  # Assuming you have only one GPU, so index is 0
print(f"GPU Name: {gpu.name}")
print(f"Total GPU Memory: {gpu.total_memory / (1024 ** 2):.2f} MB")

cuda:0
GPU Name: Tesla P100-PCIE-16GB
Total GPU Memory: 16280.69 MB


## Dataloader for self-supervised case

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

cifar_train = datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
cifar_test = datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)

train_img_array = np.array([np.array(image) for image, _ in cifar_train])
test_img_array = np.array([np.array(image) for image, _ in cifar_test])

Files already downloaded and verified
Files already downloaded and verified


In [6]:
train_dataset = ContrastiveDataset("train", train_img_array[:40000])
val_dataset = ContrastiveDataset("val", train_img_array[40000:])
#test_dataset = ContrastiveDataset("test", test_img_array)

In [7]:
ssl_batch_size = 10
num_workers = 0 # means no sub-processes, needed for debugging
train_dataloader = DataLoader(
    train_dataset, batch_size=ssl_batch_size, shuffle=True, num_workers=num_workers
)
val_dataloader = DataLoader(
    val_dataset, batch_size=ssl_batch_size, shuffle=False, num_workers=num_workers
)
#test_dataloader = DataLoader(
 #   test_dataset, batch_size=ssl_batch_size, shuffle=False, num_workers=num_workers
#)

In [8]:
model = ContrastiveLearningModel().to(device)
optimizer = LARS(
    [params for params in model.parameters() if params.requires_grad],
    lr=0.2,
    weight_decay=1e-6,
    exclude_from_weight_decay=["batch_normalization", "bias"],
)
criterion = NT_Xent(batch_size=ssl_batch_size, temperature=0.5)

In [9]:
num_epochs = 5

training_losses = []
validation_losses = []
for epoch in range(num_epochs): 

  start = time.time()
  model.train()
  training_loss = 0
  for (x_i, x_j) in train_dataloader: 
    optimizer.zero_grad()
    x_i, x_j = x_i.to(device), x_j.to(device)

    z_i = model(x_i)
    z_j = model(x_j)

    loss = criterion(z_i, z_j)
    loss.backward()

    optimizer.step()
    training_loss += loss.item()
  
  training_loss /= len(train_dataloader)
  training_losses.append(training_loss)
  
  model.eval()
  with torch.no_grad(): 
    validation_loss = 0
    for (x_i, x_j) in val_dataloader: 
      x_i, x_j = x_i.to(device), x_j.to(device)

      z_i = model(x_i)
      z_j = model(x_j)

      loss = criterion(z_i, z_j)
      validation_loss += loss.item()

    validation_loss /= len(val_dataloader)
    validation_losses.append(validation_loss)
    
    end = time.time()
    
  print(f"Epoch #{epoch+1}, training loss: {training_loss}, validation loss: {validation_loss}, time: {end - start:.2f}")

/cluster/work/jssaethe/tdt05_ssl/SSL_project/optimizer.py:132: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1025.)
  next_v.mul_(momentum).add_(scaled_lr, grad)


Epoch #1, training loss: 1.4723989049494266, validation loss: 1.376311990737915, time: 604.14


KeyboardInterrupt: 

In [ ]:
x = np.linspace(1, len(training_losses), len(training_losses))
plt.plot(x, training_losses, label="Training")
plt.plot(x, validation_losses, label="Valdation")

plt.xlabel("Num epochs")
plt.ylabel("Loss")
plt.title("Contrastive Loss")
plt.legend()

plt.show()

In [ ]:
# Saving the model to file
model_path = "models/encoder.pth" 
torch.save(model.encoder, model_path)